# <center>TP2</center>
# <center>Grupo 11</center>

#Integrantes:
- Blas Chuc - 110253
- Franco Rodriguez - 108799
- Helen Chen - 110195
- Tomas Caporaletti - 108598

##Importar librerías

In [25]:
!pip install unidecode

In [26]:
#Básico
import pandas as pd
import seaborn as sns
import numpy as np

##Modelo
import unidecode
import re
import collections
from collections import Counter, OrderedDict
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences

##Carga del dataset de trabajo

In [27]:
df = pd.read_csv('train.csv')

In [28]:
criticas = df['review_es'].values

df['sentimiento'] = df['sentimiento'].map({'negativo': 0, 'positivo': 1})
sentimientos = df['sentimiento']

In [29]:
df_test = pd.read_csv('test.csv')

In [30]:
criticas_test = df_test['review_es'].values

##Analisis de los datos

Nos encontramos frente a un dataset de train con 50.000 registros de críticas cinematográficas, donde utilizaremos un 20% para validación en nuestro futuro entrenamiento.

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           50000 non-null  int64 
 1   review_es    50000 non-null  object
 2   sentimiento  50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


Por otro lado, tenemos 8.599 registros que deberemos predecir de forma binaria ("positivo" o "negativo") con el fin de subir a nuestra competencia de Kaggle acorde al TP2 de la materia.

In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599 entries, 0 to 8598
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         8599 non-null   int64 
 1   review_es  8599 non-null   object
dtypes: int64(1), object(1)
memory usage: 134.5+ KB


##Red Neuronal

###Funciones generales

Definimos las funciones que utilizaremos tanto para entrenamiento como para test, ya que son fundamentales para el desarrollo normal de la red.

Primero y principal tenemos la limpieza y normalización de nuestras críticas, la cual es fundamental para obtener un vocabulario funcional y completo sobre este problema.
También nos permitirá dejar en las mismas condiciones las críticas que luego predeciremos, del archivo de Test.

In [33]:
def limpieza_de_critica(text):
    text = unidecode.unidecode(text.lower().strip())
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

Una vez limpias nuestras críticas, el siguiente paso importante es realizar el traspaso de arreglos de tokens (palabras en nuestro caso), a arreglos de números (por cada una de las críticas) en base a un diccionario previamente creado. Con el fin de lograr que nuestro modelo entienda la distinción entre cada uno de lo tokens y en su primer capa de Embedding, pueda ubicar cada una en una posición relativa en un espacio N dimensional.

In [34]:
def vectorizar_texto(tokens_list, word_dict):
    corpus_indices = []
    for tokens in tokens_list:
        indices = [word_dict.get(word, 1) for word in tokens]
        corpus_indices.append(indices)
    return corpus_indices

Esta función cumple el mismo propósito que la métrica de F1, pero corrigiendo problemas obtenidos durante los primeros entrenamientos, acerca de la dimensionalidad de los resultados de salida entre los valores verdaderos y los predichos.

In [35]:
def f1_score_custom(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')

    # Definir umbral
    threshold = 0.5
    y_pred_binary = K.cast(K.greater(y_pred, threshold), 'float32')

    tp = K.sum(y_true * y_pred_binary, axis=0) #True Positive
    fp = K.sum((1 - y_true) * y_pred_binary, axis=0) #False Positive
    fn = K.sum(y_true * (1 - y_pred_binary), axis=0) #False Negative

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)

###Entrenamiento

Aplicando las funciones generales en un mismo bloque necesario para obtener valores pertinentes para nuestro modelo.
Los valores serán:
- Vocab_size = indica el tamaño de nuestro vocabulario utilizado.
- Max_len = definirá cual es el tamaño de crítica que utilizaremos, rellenando con padding a aquellas que sean más cortas que dicho valor y truncando las que tengan una longitud mayor.

In [36]:
criticas_tokens = [limpieza_de_critica(c) for c in criticas]

all_words = [word for tokens in criticas_tokens for word in tokens]
word_counts = Counter(all_words)

min_count = 2
vocabulario_filtrado = [w for w, c in word_counts.items() if c >= min_count]

word2idx = {word: i + 2 for i, word in enumerate(vocabulario_filtrado)}

corpus_train = vectorizar_texto(criticas_tokens, word2idx)

longitudes = [len(x) for x in corpus_train]
MAX_LEN = int(np.percentile(longitudes, 95))
print(f"Max Len fijado en: {MAX_LEN}")

X_train = pad_sequences(corpus_train, maxlen=MAX_LEN, padding='post', truncating='post')
y_train = np.array(sentimientos).reshape(-1, 1)

vocab_size = len(word2idx) + 2

Max Len fijado en: 622


Los valores definidos arbitrariamente para el entrenamiento del modelo, surgen de reiteradas pruebas con diferentes valores, tanto para "embedding_dim", "LSTM_UNITS", "dropout", "dense", "learning_rate", "epochs", "patience" y "batch_size". Todo esto está detallado dentro del pertinente informe del TP.

In [37]:
embedding_dim = 128
LSTM_UNITS = 64

model = keras.Sequential([
    layers.Embedding(input_dim=vocab_size,
                     output_dim=embedding_dim,
                     input_length=MAX_LEN,
                     mask_zero=True),

    layers.Dropout(0.4),

    layers.Bidirectional(layers.LSTM(LSTM_UNITS, return_sequences=False)),

    layers.Dropout(0.4),

    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

optimizador = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizador,
              loss='binary_crossentropy',
              metrics=[f1_score_custom, "accuracy", "precision", "recall"])

# EarlyStopping detiene el entrenamiento si no mejora, guardando el mejor modelo
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    epochs=4,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stop])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1216s 968ms/step - accuracy: 0.5949 - f1_score_custom: 0.5576 - loss: 0.6443 - precision: 0.5944 - recall: 0.5663 - val_accuracy: 0.8486 - val_f1_score_custom: 0.8289 - val_loss: 0.3604 - val_precision: 0.9220 - val_recall: 0.7621
Epoch 2/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1221s 967ms/step - accuracy: 0.9027 - f1_score_custom: 0.8988 - loss: 0.2547 - precision: 0.9013 - recall: 0.9028 - val_accuracy: 0.8875 - val_f1_score_custom: 0.8806 - val_loss: 0.2695 - val_precision: 0.9142 - val_recall: 0.8556
Epoch 3/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1212s 970ms/step - accuracy: 0.9425 - f1_score_custom: 0.9405 - loss: 0.1632 - precision: 0.9441 - recall: 0.9396 - val_accuracy: 0.8961 - val_f1_score_custom: 0.8930 - val_loss: 0.2828 - val_precision: 0.8970 - val_recall: 0.8953
Epoch 4/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1222s 970ms/step - accuracy: 0.9622 - f1_score_custom: 0.9611 - loss: 0.1145 - precision: 0.9623 - recall: 0.9621 - val_accuracy: 0.8973 - val_

###Test

Con el modelo entrenado, estamos aptos para poder predecir nuestro archivo de Test y poder retornar una valoración para cada una de las críticas que debemos exponer en Kaggle.

In [38]:
def predecir_nuevas_reviews(reviews_test, word2idx, max_len, model):
    tokens_test = [limpieza_de_critica(r) for r in reviews_test]

    corpus_test = vectorizar_texto(tokens_test, word2idx)

    input_test = pad_sequences(corpus_test, maxlen=max_len, padding='post', truncating='post')

    preds = model.predict(input_test)
    return (preds >= 0.5).astype(int)

In [39]:
pred = predecir_nuevas_reviews(criticas_test, word2idx, MAX_LEN, model)

269/269 ━━━━━━━━━━━━━━━━━━━━ 50s 183ms/step


##Exportar solución

In [40]:
def save_prediction(pred_test, file_name):
  ids = df_test['ID'].values.tolist()
  sent_pred = ['positivo' if p == 1 else 'negativo' for p in pred_test]

  df_exportar = pd.DataFrame({
    "ID": ids,
    "sentimiento": sent_pred
  })

  df_exportar.to_csv(f"{file_name}.csv", index=False, encoding="utf-8")

In [41]:
file_name = "prediccion19"
save_prediction(pred, file_name)